In [14]:
import numpy as np

具体 Q-learning 算法, 详见 PPT或者下图:
![Algo](Qlearn_algo.png)

In [32]:
def initR(mat_size, edges):

    mat=np.full((mat_size, mat_size), fill_value=-1, dtype=np.int16)
    edges=np.array(edges, dtype=np.int16)
    for (i, j, k) in edges:
        if k==0:
            mat[i, j]=mat[j, i]=k
        else:
            mat[i, j]=k
            mat[j, i]=0
    return mat

'''
Q-Learning 强化学习代码.
R: 状态转移的奖励惩罚矩阵(看重短期收益)
Q: 状态转移的汇报矩阵(看重中长期收益)
下面那种更新算法的本质, 就是在平衡短期收益和长期收益
'''

def updateQ(matQ, matR, cnt:int, gamma:float, goal_id:int):  # 迭代

    if cnt<0:
        print(f'wrong input of cnt as cnt=={cnt} < 0')
        return None
    
    mat_size=np.shape(matQ)[0]

    for i in range(0, cnt):

        # 随机选择当前状态 x, 满足:
        # x!=goad_id(x 不是目标点, 若是的话没必要迭代) 
        # x!=y(不是自己原地绕圈)
        # y 的选取: Q(x, action)中最大的 action, 就是 y.

        '''
        但是实际在下面两个例子中, 状态转移太稀疏了.
        如果真的随便选, 反而会出现 matQ 更新完上前轮还是 全零矩阵的情况.
        因为y=np.argmax(matQ[x, :])=0, 而唯一和 0(A) 有连接的点E, 权值 R(A, E)=0, 下面的公式中 matQ[A, E]=0.
        所以这样的话, 就一直没有点更新.

        但是恰恰这样随便选, 恰好能够和 example1 的结果相同
        '''

        # matR[x, y]!=-1 (x到 y 有路径)
        
        while True:
            x = np.random.randint(0, mat_size)
            y = np.random.randint(0, mat_size)
            # y=np.argmax(matQ[x, :])
            if x!=goal_id and x != y and matR[x, y] != -1:
                break

        # 对其进行初始化
        edge_list=[]
        for z in range(0, mat_size):
            if matR[y, z]>=0:
                edge_list.append(matQ[y, z])
        if len(edge_list)>0: # 有 y->z 的路径, 且满足上满选 x, y 的条件才迭代
            edge_list=np.array(edge_list)   
            matQ[x, y] = matR[x, y]+ gamma * np.max(edge_list)

下面是第一个例子:  PPT P13
![PPT_P13](example1.png) 

In [33]:
vertexNum=6
edges=[(0, 4, 0), (3, 4, 0), (1, 3, 0), (2, 3, 0), (1, 5, 100), 
       (4, 5, 100), (5, 5, 100)] 
# 元素是三元数组(i,j,k), 分别记录了房间 i 到 j 的权值 k
# 顶点映射: A-0, B-1, C-2, D-3, E-4 F-5
# 其中, 只有B->F(1->5) 和 F->F(5->5) 和 E->F(4->5) 权值为 100, 因为这是能够到达最终目标(F)的
# 至于为什么设成 100? 这是因为 PPT P13 是这么设置的

# 建 R 矩阵的时候, 一定要把(5, 5, 100)写进 edges. 要不最终会发现 matQ 矩阵自始至终是零矩阵, 没得到过更新.
mat_Reward=initR(vertexNum, edges)

cnt=500
gamma=0.8

mat_Q=np.zeros((vertexNum, vertexNum), dtype=np.float32)
# print(mat_Q)
updateQ(mat_Q, mat_Reward, cnt, gamma, goal_id=5)

np.set_printoptions(
    precision=3,          # 保留2位小数（可根据需求调整为1或3）
    linewidth=1000,       # 单行最大宽度（设大些，避免自动换行）
    suppress=True         # 禁用科学计数法（避免出现1.2e+02这类格式）
)

print(mat_Q)

[[  0.    0.    0.    0.   80.    0. ]
 [  0.    0.    0.   64.    0.  100. ]
 [  0.    0.    0.   64.    0.    0. ]
 [  0.   80.   51.2   0.   80.    0. ]
 [ 64.    0.    0.   64.    0.  100. ]
 [  0.    0.    0.    0.    0.    0. ]]


而 PPT 上也给出了最终 matQ 矩阵的情况, 基本符合上面的代码:
![PPT_P18](example1_ans.png)

下面是第二个例子:
![PPT_P20](example2.png)
图片来源: 强化学习PPT P20


In [34]:
vertexNum=8
edges=[(0, 4, 0), (4, 5, 0), (5, 6, 0), (1, 5, 0), 
       (1, 2, 0), (2, 6, 0), (2, 3,0), (6, 7, 0), 
       (2, 3, 100), (3, 3, 100)] 
# 元素是三元数组(i,j,k), 分别记录了房间 i 到 j 的权值 k
# 顶点映射: A-0, B-1, ... G-6, H-7
# 其中, 只有C->D(2->3) 和 D->D(3->3) 权值为 100, 因为这是能够到达最终目标(D)的
# 至于为什么设成 100? 因为前面一个例子是这么干的

mat_Reward=initR(vertexNum, edges)

cnt=500
gamma=0.8

mat_Q=np.zeros((vertexNum, vertexNum), dtype=np.float32)
# print(mat_Q)
updateQ(mat_Q, mat_Reward, cnt, gamma, goal_id=3)

np.set_printoptions(
    precision=3,          # 保留2位小数（可根据需求调整为1或3）
    linewidth=1000,       # 单行最大宽度（设大些，避免自动换行）
    suppress=True         # 禁用科学计数法（避免出现1.2e+02这类格式）
)

print(mat_Q)

[[  0.      0.      0.      0.     40.96    0.      0.      0.   ]
 [  0.      0.     80.      0.      0.     51.2     0.      0.   ]
 [  0.     64.      0.    100.      0.      0.     64.      0.   ]
 [  0.      0.      0.      0.      0.      0.      0.      0.   ]
 [ 32.768   0.      0.      0.      0.     51.2     0.      0.   ]
 [  0.     64.      0.      0.     40.96    0.     64.      0.   ]
 [  0.      0.     80.      0.      0.     51.2     0.     51.2  ]
 [  0.      0.      0.      0.      0.      0.     64.      0.   ]]
